In [1]:
from sympy import *
import operator
import functools
import math
import pandas as pd
init_printing()
#init_printing(use_latex='mathjax')

In [254]:
import numpy as np
class incidence_matrix(np.ndarray):  
    #todo allow a dictionary of letters maybe for vertices to make it easier to init and maybe consider not using sizes
    def __new__(self, edge_count, vertex_count, edges=[],external_vertices=[]):
        size = (vertex_count+ (1 if len(external_vertices) > 0 else  0),edge_count+len(external_vertices))
        obj = super(incidence_matrix, self).__new__(self, size, np.int)  
        obj[:] = 0 
        obj.add_edges(edges + [[-1,ev] for ev in external_vertices])
        obj._external_vertices = external_vertices
        obj._internal_vertices = list(set(list(range(vertex_count))) - set(external_vertices))
        self._edges =edges
        return obj
            
    def flip_edges(self,edge_indecies):
        if not isinstance(edge_indecies,list): edge_indecies =[edge_indecies]
        for e in edge_indecies:self[:,e]*= -1
        return self
        
        
    def reset_edge_orders(self, edges):pass
    def reset_edge_orders_from_hierarchy(self):pass
    
    def try_directed_edge_index(self,a,b):
        """
        checks all the columns (edges) to see if there is an edge that starts at a(-1) and endgs at b(1)
        """
        for i,r in enumerate(self.T):
            if(r[a]==-1) and (r[b]==1):
                return i
        return None
    
    def exists_directed_edge(self,a,b):
        return self.try_directed_edge_index != None
    
    @property
    def edges(self): return self._edges
        
    @property
    def internal_vertices (self):return self._internal_vertices
    
    @property
    def external_vertices (self):return self._external_vertices
    
    def get_coincident_vertices_any(self, vset,exclusions = []):
        items = []
        for v in vset:
            for _v in self.get_coincident_vertices(v,exclusions):
                if _v not in items: items.append(_v)
        return items
        
    def get_coincident_vertices(self, v,exclusions = []):
        #these are the data columns for which v has edge data
        v_inc_frame = self[:,np.where(self[v]!=0)[0]]
        exclusions = exclusions+ [v]
        #now get the row index for things that are co-incident using the trick but just checking non zero rows
        #this is a bit odd and probably better way but count non zero columns uses the count and nonzero combined 
        #and we ensure that v is not in the list which is requred when it has only one edge for example
        incident_vertices = [l for l in list(np.nonzero((np.count_nonzero(v_inc_frame,axis=1)==1).astype(int))[0]) if l not in exclusions]
        return incident_vertices
        
    def add_edges(self, eset):
        for i, v in enumerate(eset): 
            self[v[0],i],self[v[1],i] = 1,-1
            
    def vertex_hierarchy(self, vset=[-1], d=0, h={} ):
        """
        recursion from external nodes or other seed set
        """
        if d ==0: h= {}
        #need to re-think the indexing a little
        #this is so that I can store the last vertex (inf) as either -1 or the length of the thing
        if -1 in vset:h[len(self)-1] = 2
        for _v in vset: h[_v] = d
        if len(vset) > 0:yield d, vset
        #look for any coincident vertices that are not on my level or above
        next_layer = self.get_coincident_vertices_any(vset ,vset+ list(h.keys()))
        if(len(next_layer)>0):
            for t in self.vertex_hierarchy(next_layer,d+1,h):
                yield t
    
    
    @property
    def flat_adjacency(self): return [ list(np.nonzero(v)[0]) for v in  self.adjacency_with_inf]
      
    @property
    def adjacency(self):
        #[ np.nonzero(v)[0]for v in  INC.adjacency_with_inf]
        return self.adjacency_with_inf[:-1,:-1]
    
    @property
    def adjacency_with_inf(self):
        ad = np.zeros((len(self),len(self)),np.int)
        for i, v in enumerate(self):  ad[i][self.get_coincident_vertices(i)]  =1
        return ad[:-1,:-1] if len(self._external_vertices) > 0 else ad
    

    @property
    def edge_labels(self):
        #the last row is the external vertex so check which edges are connected
        ex_edges = np.nonzero(self[-1])[0]
        return [ "p"+str(i) for i in range(self.shape[-1]) if i not in ex_edges]   +   [ "q"+str(i) for i in range(len(ex_edges))]
       
    @property
    def vertex_constraints(self):
        edge_labels = self.edge_labels
        for v in self:
            yield delta_constraint([edge_labels[i] for i in np.nonzero(v)[0]])
            
INC = incidence_matrix(9, 6,
                       [ [0,1], [1,2], [2,0], [3,4],[4,3],[3,0],[4,2], [4,5], [5,3] ],
                       external_vertices=[0,1,2])

INC.edge_labels

['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'q0', 'q1', 'q2']

In [239]:
class cycle_finder(object):
    """
    Adapted from
    https://stackoverflow.com/questions/12367801/finding-all-cycles-in-undirected-graphs
    See also
    http://dspace.mit.edu/bitstream/handle/1721.1/68106/FTL_R_1982_07.pdf
    #graph = [[1, 2], [1, 3], [1, 4], [2, 3], [3, 4], [2, 6], [4, 6], [8, 7], [8, 9], [9, 7]]
    graph = [[0,1], [0,2], [1,2], [1,3], [2,3]]

    for p in cycle_finder(graph): print(p) #returns the edges in the circuit indexed from the graph
    for p in cycle_finder(graph, "vertices"): # returns the vertices in the cicuit

    """
    def __init__(self, graph_edges, c_type="edges"):
        #i need them sorted here - this is an undirected graph too
        self.graph = [sorted(l) for l in list(graph_edges)]
        self.cycles = []  
        self.c_type = c_type
        for edge in self.graph:
            for node in edge:  
                self.findNewCycles([node])
    
    def __expand_edges__(self, vlist):
        for i,v in enumerate(vlist): 
            yield sorted([vlist[i-1],v])
            
    @property
    def cycle_basis(self):
        l = list(self)
        b = np.zeros((len(l),len(self.graph)),np.int)
        for i,r in enumerate(b): r[l[i]] = 1
        return b
    
    def __iter__(self):
        for cy in self.cycles:
            res = [node for node in cy]
            if self.c_type == "vertices": yield res
            else: yield [self.graph.index(e) for e in self.__expand_edges__(res)]
               
    def findNewCycles(self,path):
        def isNew(path): return not path in self.cycles
        def visited(node, path):  return node in path
        def invert(path):return rotate_to_smallest(path[::-1])
        def rotate_to_smallest(path):
            n = path.index(min(path))
            return path[n:]+path[:n]   
        
        start_node,next_node,sub = path[0],None,[]
        
        for edge in self.graph:
            node1, node2 = edge
            if start_node in edge:
                    next_node = node2 if node1 == start_node else node1
            if not visited(next_node, path):
                    sub = [next_node]
                    sub.extend(path)
                    self.findNewCycles(sub);
            elif len(path) > 2  and next_node == path[-1]:
                    p = rotate_to_smallest(path);
                    inv = invert(p)
                    if isNew(p) and isNew(inv):  
                        self.cycles.append(p)
                        
                        
graph = [[0,1], [0,2], [1,2], [1,3], [2,3]]
for p in cycle_finder(graph): print(p)
f = cycle_finder(graph)
f.cycle_basis

[0, 1, 2]
[0, 1, 4, 3]
[2, 3, 4]


array([[1, 1, 1, 0, 0],
       [1, 1, 0, 1, 1],
       [0, 0, 1, 1, 1]])

In [66]:
cf = cycle_finder(INC.edges) 
list(cf)

In [67]:
cf.cycle_basis

array([[1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 1, 1, 0, 0],
       [1, 1, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 0, 1, 1]])

In [68]:
def _xor_add(u,v):return np.logical_xor(u,v).astype(int)

In [69]:
_xor_add(cf.cycle_basis[0], cf.cycle_basis[1])

array([0, 0, 1, 1, 0, 1, 1, 0, 0])

In [240]:
D = ring_diagram(INC) 
D

probing for 0 1
flip 0 1
1 0
probing for 1 2
flip 1 2
2 1
probing for 2 0
flip 2 3
3 2
probing for 3 4
0 1
probing for 4 3
1 2


In [241]:
INC

incidence_matrix([[ 1,  0, -1,  0,  0, -1,  0,  0,  0, -1,  0,  0],
                  [-1,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0],
                  [ 0, -1,  1,  0,  0,  0, -1,  0,  0,  0,  0, -1],
                  [ 0,  0,  0,  1, -1,  1,  0,  0, -1,  0,  0,  0],
                  [ 0,  0,  0, -1,  1,  0,  1,  1,  0,  0,  0,  0],
                  [ 0,  0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0],
                  [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1]])

In [ ]:
#wishlist
#add edge labels to edges on the graph sometimes for comparison
#mark edges by mask

In [196]:
class arc(object):
    def __init__(self, angles, radius,center=None,size=None,pid=None):
        self.pid = pid
        self.radius = radius
        object_radius = 20 + radius
        self.center = (object_radius,object_radius) if center is None else center
        self.size=(2*object_radius,2*object_radius) if size is None else size
        self.body = ""
        self.header =  arc.get_header()
        
        startAngle,endAngle = angles[0],angles[1]    
        mid_angle = (startAngle+endAngle)/2.
        
        #self.body+= """<circle cx="{0}" cy="{1}" r="2" stroke="black" stroke-width="1" fill="red" /> """.format(*self.center)
            
        self.body+= self.__describeArc__(startAngle, endAngle)
        self.body+= self.__describeArc__(startAngle, mid_angle, decorations="marker-end='url(#arrow)'")
           
    @staticmethod
    def get_header():
                 return """<defs><marker id="arrow"  markerUnits="strokeWidth" markerWidth='5' markerHeight='8' refX='0' refY='2' orient="auto">
                                      <path d="M0,0 V4 L2,2 Z" style="fill: #000000;" /> </marker></defs>"""
     
    
    def __polarToCartesian__(self, angleInDegrees, radius, center):
        centerX, centerY = center[0], center[1]
        angleInRadians = (angleInDegrees-90) *math.pi / 180.0;
        return int( centerX + (radius * math.cos(angleInRadians))), int( centerY + (radius * math.sin(angleInRadians))), angleInDegrees
          
    def __describeArc__(self, startAngle, endAngle, decorations=""):
        radius = self.radius
        wrap,sweepFlag = (0,0) if endAngle < startAngle  else (0,1)

        start = self.__polarToCartesian__(startAngle+wrap, radius, self.center) 
        end = self.__polarToCartesian__(endAngle+wrap, radius, self.center)
        colour = "black"
        largeArcFlag = "0" if endAngle - startAngle <= 180 else "1"
        #print(startAngle+wrap,endAngle+wrap, largeArcFlag, sweepFlag)
        d = [  "M", start[0], start[1], "A", radius, radius, 0, largeArcFlag, sweepFlag, end[0], end[1]  ]
        d = " ".join([str(_d)+" " for _d  in d] )   
        
        transfrm =  ""#  """transform="rotate(180, {}, {}) scale(1, -1) translate(0, -150)" """.format(*self.center) if self.change_sense else ""
        pidattr = "" if self.pid == None else """id='edge{}'""".format(self.pid)
        
        return """<path {} d="{}" stroke="{}"  stroke-width:3px" {} {}/>""".format(pidattr, d,colour, decorations, transfrm) 
    
    def __repr__(self):return self.header + self.body
    
    def _repr_html_(self): 
        return   """<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="{1}" y="{2}" width="{3}" height="{4}">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> {0}  </g> </svg>""".format(self.__repr__(),  *self.center, *self.size)

#arc((130,250),radius=50,change_sense=False)

In [271]:
class ring_diagram(object):
    """
    This one uses a concentric circle arrangement starting from nodes connected to v_\{infty}
    This is a rough prototype to play around with the SVG
    Actually this needs a better integration with the incident matrix i.e. the incident matrix needs some concepts
    that are used in drawing such as drawing callbacks for arcs and edges mapped to a coodinate system
    then we should be able to enumerate through all arcs and edges in the diagram
    that would make it easy to show only one edge w.r.t the origin and be alot more functional
    important thing is to make sure the drawing is driven from the incidence matrix 
    this avoids the problem of connecting out to the outer rings which was just a test run
    the incident matrix then needs to be validated by independent cycles
    """
    def __init__(self, inc_matrix, options={}):
        self.x = 0
        self.y = 0
        self.margin = 10
        self.radius = 90
        center = self.radius+self.margin
        self._size=(200,200)
        self._center = (center,center)
        self._options = options
        self._inc = inc_matrix
        self._vertex_depths = [v[1] for v in list(self._inc.vertex_hierarchy())][1:]
        self._metadata = {}
        self.body = ""
        
    @property
    def external_vertices(self):mpass

    def __circuit_permutations__(self):
        #so for all adjacency matrix entries, permute so as to choose a particular cycle set
        #oursource to algorithm
        yield None
        
    def __circuit_edge_permutations__(self):
        #for each cycle set, choose the independent cylces by fixing an edge delegate in each independent cicuit
        yield None
        
    def __measure_inter_layer_chords__(self, existing_coords, proposed_coords):
        return 0
    
    def __polarToCartesian__(self, angleInDegrees, radius, center):
        centerX, centerY = center[0], center[1]
        angleInRadians = (angleInDegrees-90) *math.pi / 180.0;
        return int( centerX + (radius * math.cos(angleInRadians))), int( centerY + (radius * math.sin(angleInRadians))), angleInDegrees
    
    def __set_vertex_property__(self, i, key, value):
        if i not in self._metadata: self._metadata[i] = {}
        self._metadata[i][key] = value
        
    def __get_vertex_property__(self, i, key,value):
        if i in self._metadata and key in self._metadata[i]:return self._metadata[i][p]
    
    def __arrange__(self, angle, radius, vset, center=None, prev_layer_connections={}):
        center = center if center is not None else self._center
        if angle == 360: 
            self.__set_vertex_property__(vset[0], "point", center)
            return [center]#if the angle is 360 this can only be a dot in the center
        m,accepted = 1,None
        for offset in [angle]:
            proposed = [self.__polarToCartesian__( (i+1)*angle, radius,center) for i, v in enumerate(vset)]
            _m = self.__measure_inter_layer_chords__(prev_layer_connections, proposed)
            if m ==-1 or _m < m: 
                accepted, m = proposed, _m
                for i,v in enumerate(vset):
                    self.__set_vertex_property__(v, "point", [proposed[i][0],proposed[i][1]])
                    self.__set_vertex_property__(v, "angle", proposed[i][-1])
                    self.__set_vertex_property__(v, "radius", radius)
                            
        return accepted

    def __describeArc__(self, startAngle, endAngle, radius,pid=None):
        return arc((startAngle,endAngle),radius, self._center,self._size, pid=pid).body    
           
    def try_get_connecting_edges(self, v):
        #look up the metadata for v and get all edges that are adjacent to it that we know about
        for u in self._inc.get_coincident_vertices(v):
            if u in self._metadata and v in self._metadata: 
                _u,_v = u,v
                #if the max length is exceeded, try to get a nice arc otherwise....
                #max length is the length of the verticle chord connecting the bottom of one disk to the point on the outer disk          
                pid = self._inc.try_directed_edge_index(v,u)
                if pid == None: 
                    _u,_v = _v,_u
                    pid = self._inc.try_directed_edge_index(v,u)
                
                yield self.get_line([self._metadata[_v]["point"][0],
                                     self._metadata[_v]["point"][1],
                                     self._metadata[_u]["point"][0],
                                     self._metadata[_u]["point"][1]],
                                     pid=pid)
        
    def get_line(self, pts, colour="black", dash_array=None,pid=None):
        dash_array = """stroke-dasharray='{}'""".format(dash_array) if dash_array != None else ""
        midpoint = [(pts[0] + pts[2])/2.,(pts[1] + pts[3])/2.]            
        #l = """<line x1="{0}" y1="{1}" x2="{2}" y2="{3}" stroke="{4}" stroke-width:2px"  {5}   />""".format(*pts, colour,dash_array )
        pidattr = "" if pid == None else """id='edge{}'""".format(pid)
        l = """<path d="M{0} {1} L {2} {3}" stroke="{4}" stroke-width:2px" {5} {6} /> """.format(*pts, colour,dash_array,pidattr )
        hl = """<line x1="{0}" y1="{1}" x2="{2}" y2="{3}" stroke="{4}" stroke-width:2px"  {5}  marker-end='url(#arrow)' />""".format(pts[0], pts[1], midpoint[0], midpoint[1], colour,dash_array )
        return l+hl
    
    def __edge_segments__(self,n,a,b,vset):
        """
        We always arrange vertices on disks separated by angles. Can only have vertices in [0,360]
        We assume that the id is the arrangment id in order and that a<b for normal sense
        if however b<a, then we have anticlockwise sense and we will change the angles for downstream reflection
        """
        #if there are multiple edges, then allow - otherwise restrict direction - wrapping because its a ring
        pid = self._inc.try_directed_edge_index(vset[a%len(vset)],vset[b%len(vset)])
        if pid == None:
            a,b, = b,a
            #try and get a backup pid - i need to thing about my sustem though
            pid = self._inc.try_directed_edge_index(vset[a%len(vset)],vset[b%len(vset)])

        #get the edge index in the incidence matrix too - in fact we could just do that and if it doesnt exist we have above
        cap = lambda x : 0 if x == 360 else x
        slices = 360./n
        first = a*slices
        second = b*slices
        if a< b: return (a*slices,b*slices), pid
        return (a*slices,cap(b*slices)), pid

    def __display__(self,edge_colouring = None, edge_styles = None):
        ##arrangement algorithm
        #each level of the hiearchy is given a radius >= 0 and maximally seperated items are added
        #there are symmetric locations and these are chosen to minimise arc lengths
        #I think maybe just try rotations of some integer factors of the freedom which is 360/|l| not including the one that gives the symmetry of course      
        marker_style = arc.get_header()
        self.body += marker_style
        levels = len(self._vertex_depths)
        
        radii_margins = [20*(levels-(i)) for i in reversed(range(len(self._vertex_depths))) ]
        angles = [(360./len(v)) for v in self._vertex_depths]
        for i, vset in enumerate(self._vertex_depths): 
            radius = self.radius-radii_margins[i]
            for coord in self.__arrange__(angles[i],radius , vset):
                self.body+= """<circle cx="{0}" cy="{1}" r="3" stroke="black" stroke-width="1" fill="black" /> """.format(*coord)
                
            #above the vertices can be aligned in any sensible way but below, edges must respect direction of incidence   
            #refactor below to simply go to the incidence matrix and get edges... if they are same-loop, arc them, otherwise straight line?
            
            for j,v in enumerate(vset):#make loops
                if len(vset) == 1: continue
                edge, pid = self.__edge_segments__(len(vset),j,j+1,vset)
                self.body += self.__describeArc__(*edge, radius, pid=pid )

            if i > 0:#after the first layer, connect edges inter-layer
                for v in vset: 
                    for e in self.try_get_connecting_edges(v):  self.body += e
                        
            #for e in range(len(self._inc.edges)):
            #    self.body+= """<text x="10%" style="fill: #000000;stroke: none; font-size: 16px;" ><textPath xlink:href="#edge{0}">e{0}</textPath></text>""".format(e)
        
        return  """<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" x="{1}" y="{2}" width="{3}" height="{4}">
      <g fill="none" stroke="black" stroke-width="1.6" stroke-linecap="round"> {0}  </g> </svg>""".format(self.body,  self.x, self.y, *self._size)

    #def __repr__(self):return self.__display__()           
    def __div2s__(self,n):
        while n - int(n) == 0:
            n/=2
            if n - int(n) ==0: yield n

                
    def _repr_html_(self): return self.__display__()
    
    @property
    def vertex_delta_functions(self):
        pass
    
    @property
    def loop_delta_functions(self):
        pass
    
D = ring_diagram(INC) 
D

In [272]:
inc = INC.copy()
inc

incidence_matrix([[ 1,  0, -1,  0,  0, -1,  0,  0,  0, -1,  0,  0],
                  [-1,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0],
                  [ 0, -1,  1,  0,  0,  0, -1,  0,  0,  0,  0, -1],
                  [ 0,  0,  0,  1, -1,  1,  0,  0, -1,  0,  0,  0],
                  [ 0,  0,  0, -1,  1,  0,  1,  1,  0,  0,  0,  0],
                  [ 0,  0,  0,  0,  0,  0,  0, -1,  1,  0,  0,  0],
                  [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1]])

In [283]:
inc.flip_edges(8)
ring_diagram(inc) 

In [284]:
inc

incidence_matrix([[ 1,  0, -1,  0,  0,  1,  0,  0,  0, -1,  0,  0],
                  [-1,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0],
                  [ 0, -1,  1,  0,  0,  0, -1,  0,  0,  0,  0, -1],
                  [ 0,  0,  0,  1, -1, -1,  0,  0,  1,  0,  0,  0],
                  [ 0,  0,  0, -1,  1,  0,  1, -1,  0,  0,  0,  0],
                  [ 0,  0,  0,  0,  0,  0,  0,  1, -1,  0,  0,  0],
                  [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1]])